<a href="https://colab.research.google.com/github/akipi-ro/daily_study/blob/master/%E6%9C%AA%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#rstanがインストールされていないならインストールする
x<-installed.packages()
if(any(x[,1]== 'rstan') == FALSE){
    system("add-apt-repository -y ppa:marutter/rrutter3.5")
    system("add-apt-repository -y ppa:marutter/c2d4u3.5") 
    system("apt-get update")
    system("apt install -y r-cran-rstan")
}

SyntaxError: ignored

In [ ]:
library(rstan)
# Automobile Datasetより
auto <- read.csv("Automobile.csv") # データフレームが得られる 
weight.gas <- auto[auto$fuel.type=="gas",]$curb.weight # ガソリン車の行から車両重量を取り出す
weight.diesel <- auto[auto$fuel.type=="diesel",]$curb.weight # ディーゼル車の行から車両重量を取り出す
# 箱ひげ図を描く


summary(weight.gas)
summary(weight.diesel)


mean(weight.gas)
mean(weight.diesel)
sd(weight.gas)#標準偏差
sd(weight.diesel)
var(weight.gas)#分散
var(weight.diesel)

boxplot(auto$curb.weight ~ auto$fuel.type)


library(rstan)
#ダイエット問題
x1 <- c(53.1,51.5,45.5,55.5,49.6,50.1,59.2,54.7,53.0,48.6,55.3,52.6,51.7,
        48.6,56.4,42.9,50.3,42.4,51.2,39.1)
x2 <- c(48.3,45.2,46.6,56.6,41.2,44.6,51.9,55.5,45.4,47.6,50.6,54.5,49.0,
        43.9,53.8,40.1,52.8,35.3,55.6,38.0)
n <- length(x1)
diet <- matrix(c(x1,x2),n,2)

#標準偏差が共通した2変量正規分布モデル
fit <- stan("2groups-paired.stan",
            data=list(n = n,
                      x = diet),
            iter=21000,warmup=1000,chains=5)
stan_hist(fit,pars=c("mu[1]","mu[2]","sigma","rho"))
print(fit,probs=c(0.025,0.05,0.95,0.975))

#標準偏差が異なる2変量正規分布モデル
fit <- stan("2groups-paired-DEF.stan",
            data=list(n = n,
                      x = diet),
            iter=21000,warmup=1000,chains=5)
print(fit,probs=c(0.025,0.05,0.95,0.975))




In [ ]:

data {
  int n; // 各群のデータ数
  vector[2] x[n]; // 第1群、第2群のデータ
}

parameters {
  vector[2] mu; // 平均ベクトル μ
  real<lower=0> sigma;  // 第1群、第2群の標準偏差σ
  real<lower=-1,upper=1> rho; // 相関係数ρ
}

transformed parameters {
  cov_matrix[2] Sigma; // 分散共分散行列Σ
  Sigma[1,1]  = sigma ^ 2;
  Sigma[2,2]  = sigma ^ 2;
  Sigma[1,2]  = rho * sigma ^ 2;
  Sigma[2,1]  = rho * sigma ^ 2;
}

model {
  mu ~ uniform(0,100);
  sigma ~ uniform(0,50);
  rho ~ uniform(-1,1);

  x ~ multi_normal(mu,Sigma); // 多変量正規分布
}

generated quantities {
  real mu1_minus_mu2;
  real u_mu1_minus_mu2_gt_2;
  real delta;
  real u_delta_gt_03;
  real one_minus_U3;
  real u_U3_gt_06;
  real pi_d;
  real u_pi_d_gt_07;
  real sigmap;
  real deltap;
  real pip_d;
  real pip_4;
  real Con;

  mu1_minus_mu2 = mu[1] - mu[2]; // μ1-μ2 式(3.11)
  u_mu1_minus_mu2_gt_2 = mu[1] - mu[2] > 2; // u_{μ1-μ2>c} 式(3.13)
  delta = (mu[1] - mu[2]) / sigma; // δ 式(3.15)
  u_delta_gt_03 = delta > 0.3; // u_{δ>c} 式(3.16)
  one_minus_U3 = normal_cdf(mu[2], mu[1], sigma); // 1-U3
  u_U3_gt_06 = one_minus_U3 < 0.4; // u_{U3>c} 式(3.21)
  pi_d = normal_cdf(delta / sqrt(2), 0, 1); // πd 式(3.24)
  u_pi_d_gt_07 = pi_d > 0.7; // u_{πd>c} 式(3.26)
  sigmap = sigma * sqrt(2 * (1 - rho)); // σ' 式(4.23)
  deltap = (mu[1] - mu[2]) / sigmap; // δ' 式(4.26)
  pip_d = normal_cdf((mu[1] - mu[2]) / sigmap, 0, 1); // π'd 式(4.29)
  pip_4 = normal_cdf((mu[1] - mu[2] - 4.0) / sigmap, 0, 1); // π'c 式(4.32)
  Con = 0.5 + asin(rho)/pi(); // 式(4.36)
}
